# Spatial dynamics analytics
* Dynamics of spatial dependence 
* Markov-based methods
    * Classic Markov
    * Spatial Markov
    * LISA Markov

In [ ]:
import matplotlib
import numpy as np
import pysal as ps
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
pdUS_real = pd.read_csv("data/US_state_pci_constant09_1929_2009.csv")
data_table = ps.pdio.read_files(ps.examples.get_path('us48.shp'))
complete_table = data_table.merge(pdUS_real,left_on='STATE_NAME',right_on='Name')
complete_table.head()

## Dynamics of Spatial Dependence

In [ ]:
names = pdUS_real["Name"].values
years = range(1929,2010)
pd_pci_real = pdUS_real[map(str,years)]
pd_pci_real.index = names
pd_pci_real.head()

In [ ]:
pci_real = pd_pci_real.values
pci_real.shape

Prepare the spatial weight matrix - queen contiguity

In [ ]:
w = ps.open(ps.examples.get_path("states48.gal")).read()
w.transform = 'R'

In [ ]:
mits = [ps.esda.moran.Moran(cs, w) for cs in pci_real.T]
res = np.array([(mi.I, mi.EI, mi.seI_norm, mi.sim[974]) for mi in mits])
fig, ax = plt.subplots(nrows=1, ncols=1,figsize = (10,5) )
ax.plot(years, res[:,0], label='Moran\'s I')
#plot(years, res[:,1], label='E[I]')
ax.plot(years, res[:,1]+1.96*res[:,2], label='Upper bound',linestyle='dashed')
ax.plot(years, res[:,1]-1.96*res[:,2], label='Lower bound',linestyle='dashed')
ax.set_title("Global spatial autocorrelation for annual US per capita incomes",fontdict={'fontsize':15})
ax.set_xlim([1929,2009])
ax.legend()

## Markov-based methods 

### Classic Discrete Markov Chains

```python
ps.spatial_dynamics.markov.Markov(self, class_ids, classes=None)
```

$$\hat{p}_{i,j} = \frac{n_{i,j}}{\sum_{q=1}^k n_{i,q} }$$

Discretization

In [ ]:
pci_real.mean(axis=0).shape

In [ ]:
mean = pci_real.mean(axis=0)
mean.shape = (81,1)
rpci_real = (pci_real.T / mean).T
rpci_real

In [ ]:
pooled_rpci_real = rpci_real.flatten()
sns.kdeplot(pooled_rpci_real,shade=True)

In [ ]:
pooled_n = len(pooled_rpci_real)
pooled_rpci_real.sort()
plt.axvline(pooled_rpci_real[int(pooled_n * 0.2)],color="r")

In [ ]:
sns.kdeplot(pooled_rpci_real,shade=True)
plt.axvline(pooled_rpci_real[int(pooled_n * 0.2)],color="r")
plt.axvline(pooled_rpci_real[int(pooled_n * 0.4)],color="r")
plt.axvline(pooled_rpci_real[int(pooled_n * 0.6)],color="r")
plt.axvline(pooled_rpci_real[int(pooled_n * 0.8)],color="r")

In [ ]:
rq = ps.Quantiles(rpci_real.flatten(),k=5).yb
rq.shape = rpci_real.shape
rq[0]

In [ ]:
rq[1]

In [ ]:
mm = ps.spatial_dynamics.markov.Markov(rq)

In [ ]:
mm.p

In [ ]:
mm.steady_state

In [ ]:
ps.spatial_dynamics.ergodic.fmpt(mm.p)

### Spatial Markov 
- consider the impacts of regions' income levels on their neighbors in the following time period

```python
giddy.markov.Spatial_Markov(self, y, w, k=4, permutations=0, fixed=False, variable_name=None)
```

In [ ]:
smarkov = ps.spatial_dynamics.markov.Spatial_Markov(rpci_real, w, fixed = True, k = 5)

In [ ]:
ps.spatial_dynamics.markov.Spatial_Markov?

In [ ]:
smarkov.summary()

Steady state distributions

In [ ]:
smarkov.s

In [ ]:
smarkov.S

In [ ]:
smarkov.F

### LISA Markov 

* consider the joint transitions of regions' and neighbors' income levels
* Markov state space={1(HH), 2(LH), 3(LL), 4(HL)}

```python
giddy.markov.LISA_Markov(self, y, w, permutations=0, significance_level=0.05, geoda_quads=False)
```

In [ ]:
ps.spatial_dynamics.markov.LISA_Markov?

In [ ]:
lm = ps.spatial_dynamics.markov.LISA_Markov(rpci_real, w, permutations=999)
lm.classes

In [ ]:
lm.p

In [ ]:
lm.steady_state

In [ ]:
ps.spatial_dynamics.ergodic.fmpt(lm.p)

In [ ]:
lm.chi_2

### References

* Rey, S. J. 2001. “[Spatial Empirics for Economic Growth and Convergence](http://onlinelibrary.wiley.com/doi/10.1111/j.1538-4632.2001.tb00444.x/full).” Geographical Analysis 33 (3). Wiley Online Library: 195–214.
* Bickenbach, F., and E. Bode. 2003. “[Evaluating the Markov Property in Studies of Economic Convergence](http://journals.sagepub.com/doi/abs/10.1177/0160017603253789?journalCode=irxa).” International Regional Science Review 26 (3): 363–92.